In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
def risk_appetite(eqty, tolerance, mn, mx, span, shape):
    """
    :param eqty: equity curve series
    :param tolerance: tolerance for drawdown (<0)
    :param mn: min risk
    :param mx: max risk
    :param span: exponential moving average to smoothe the risk_appetite
    :param shape: convex (>45 deg diagonal) = 1, concave (<diagonal) = -1, else: simple risk_appetite
    """

    # drawdown rebased
    eqty = pd.Series(eqty)
    watermark = eqty.expanding().max()  # all-time-high peak equity
    drawdown = eqty / watermark - 1  # drawdown from peak
    ddr = 1 - np.minimum(drawdown / tolerance, 1)  # drawdown rebased to tolerance from 0 to 1
    avg_ddr = ddr.ewm(span=span).mean()  # span rebased drawdown

    # Shape of the curve
    if shape == 1:  #
        _power = mx / mn  # convex
    elif shape == -1:
        _power = mn / mx  # concave
    else:
        _power = 1  # raw, straight line
    ddr_power = avg_ddr ** _power  # ddr

    # mn + adjusted delta
    risk_appetite = mn + (mx - mn) * ddr_power

    return risk_appetite

# Portfolio Simulation
1. Hypothetical portfolio is benchmarked to the S&P 500 index
2. Initial capital (K) is set at USD 1 million
3. Beta (sensitivity to the market) has been extracted from the Yahoo Finance website
4. The number of shares and relative stop losses are calibrated to -0.50% relative risk adjusted to the portfolio
5. The portfolio is run from December 31, 2020, through June 30, 2021

In [4]:
port = np.nan
K = 1000000
lot = 100
port_tickers = ['QCOM', 'TSLA', 'NFLX', 'DIS', 'PG', 'MMM', 'IBM', 'BRK-B', 'UPS', 'F']
bm_ticker = '^GSPC'
tickers_list = [bm_ticker] + port_tickers
df_data = {
    'Beta': [1.34, 2, 0.75, 1.2, 0.41, 0.95, 1.23, 0.9, 1.05, 1.15],
    'Shares': [-1900, -100, -400, -800, -5500, 1600, 1800, 2800, 1100, 20800],
    'rSL': [42.75, 231, 156, 54.2, 37.5, 42.75, 29.97, 59.97, 39.97, 2.10]
}
port = pd.DataFrame(df_data, index=port_tickers)
port['Side'] = np.sign(port['Shares'])

start_dt = '2021-01-01'
end_dt = '2021-07-01'
price_df = round(yf.download(tickers=tickers_list, start='2021-01-01', end='2021-07-01',
                             interval="1d", group_by='column', auto_adjust=True,
                             prepost=True, threads=True, proxy=None)['Close'], 2)

bm_cost = price_df[bm_ticker][0]
bm_price = price_df[bm_ticker][-1]

port['rCost'] = round(price_df.iloc[0, :].div(bm_cost) * 1000, 2)
port['rPrice'] = round(price_df.iloc[-1, :].div(bm_price) * 1000, 2)
port['Cost'] = price_df.iloc[0, :]
port['Price'] = price_df.iloc[-1, :]

print(port)

[*********************100%***********************]  11 of 11 completed
       Beta  Shares     rSL  Side   rCost  rPrice    Cost   Price
QCOM   1.34   -1900   42.75    -1   38.33   32.09  141.86  137.89
TSLA   2.00    -100  231.00    -1   65.73   52.72  243.26  226.57
NFLX   0.75    -400  156.00    -1  141.29  122.91  522.86  528.21
DIS    1.20    -800   54.20    -1   48.01   40.90  177.68  175.77
PG     0.41   -5500   37.50    -1   35.23   30.07  130.36  129.22
MMM    0.95    1600   42.75     1   42.62   43.08  157.72  185.13
IBM    1.23    1800   29.97     1   28.69   29.94  106.18  128.68
BRK-B  0.90    2800   59.97     1   61.73   64.67  228.45  277.92
UPS    1.05    1100   39.97     1   41.58   45.96  153.86  197.53
F      1.15   20800    2.10     1    2.09    3.13    7.72   13.47


In [5]:
# BV, book value: cost in fund currency (USD) * open positions
BV = port['Shares'] * port['Cost']

# MV, Market Value: shares in currency (USD) * current close price
MV = port['Shares'] * port['Price']

# rMV, relative market value
rMV = port['Shares'] * port['rPrice']

port['rR'] = (port['rCost'] - port['rSL'])
port['Weight'] = round(MV.div(abs(MV).sum()), 3)
port['rRisk'] = -round(np.maximum(0, (port['rR'] * port['Shares']) / K), 4)
port['rRAR'] = round((port['rPrice'] - port['rCost']) / port['rR'], 1)
port['rCTR'] = round(port['Shares'] * (port['rPrice'] - port['rCost']) / K, 4)
port['CTR'] = round(port['Shares'] * (port['Price'] - port['Cost']) / K, 4)
port_long = port[port['Side'] > 0]
port_short = port[port['Side'] < 0]

concentration = (port_long['Side'].count() - port_short['Side'].count()) / port['Side'].count()

# Gross exposure: absolute sum of MV fx adjusted divided by K
gross = round(abs(MV).sum() / K, 3)

# Net exposure: arithmetic net sum of MV divided by gross exposure
net = round(MV.sum() / abs(MV).sum(), 3)

# Net beta: sum product of MV * Beta divided by gross exposure
net_Beta = round((MV * port['Beta']).sum() / abs(MV).sum(), 2)
5
print('Gross Exposure', gross, 'Net Exposure', net, 'Net Beta', net_Beta, 'concentration', concentration)

rnet = round(rMV.sum() / abs(rMV).sum(), 3)
rnet_Beta = round((rMV * port['Beta']).sum() / abs(rMV).sum(), 2)

print('rGross Exposure', gross, 'rNet Exposure', rnet, 'rNet Beta', rnet_Beta)

Gross Exposure 3.151 Net Exposure 0.145 Net Beta 0.25 concentration 0.0
rGross Exposure 3.151 rNet Exposure 0.145 rNet Beta 0.25


In [6]:
port[['Side', 'Weight', 'rRisk', 'rRAR', 'rCTR', 'CTR']].sort_values(by=['Side', 'rRAR'])

,Side,Weight,rRisk,rRAR,rCTR,CTR
TSLA,-1,-0.007,-0.0165,0.1,0.0013,0.0017
DIS,-1,-0.045,-0.0050,1.1,0.0057,0.0015
NFLX,-1,-0.067,-0.0059,1.2,0.0074,-0.0021
QCOM,-1,-0.083,-0.0084,1.4,0.0119,0.0075
PG,-1,-0.226,-0.0125,2.3,0.0284,0.0063
F,1,0.089,-0.0000,-104.0,0.0216,0.1196
MMM,1,0.094,-0.0000,-3.5,0.0007,0.0439
IBM,1,0.074,-0.0000,-1.0,0.0022,0.0405
BRK-B,1,0.247,-0.0049,1.7,0.0082,0.1385
UPS,1,0.069,-0.0018,2.7,0.0048,0.0480


In [7]:
port[['Side', 'Weight', 'rRisk', 'rRAR', 'rCTR', 'CTR']].groupby('Side').sum()

,Weight,rRisk,rRAR,rCTR,CTR
Side,,,,,
-1,-0.428,-0.0483,6.1,0.0547,0.0149
1,0.573,-0.0067,-104.1,0.0375,0.3905


In [8]:
adjust_long = adjust_short = -0.01

pro_rata_long = port_long['rRisk'] / (port_long['rRisk'].sum() * port_long['rRAR'])
risk_adj_long = (abs(adjust_long) * pro_rata_long * K / port_long['rR'] // lot) * lot
shares_adj_long = np.minimum(risk_adj_long, port_long['Shares']) * np.sign(adjust_long)

pro_rata_short = port_short['rRisk'] / (port_short['rRisk'].sum() * port_short['rRAR'])
risk_adj_short = (abs(adjust_short) * pro_rata_short * K / port_short['rR'] // lot) * lot
shares_adj_short = np.maximum(risk_adj_short, port_short['Shares']) * np.sign(adjust_short)

port['Qty_adj'] = shares_adj_short.append(shares_adj_long)
port['Shares_adj'] = port['Shares'] + port['Qty_adj']
port['rRisk_adj'] = -round(np.maximum(0, (port['rR'] * port['Shares_adj']) / K), 4)
MV_adj = port['Shares_adj'] * port['Price']
rMV_adj = port['Shares_adj'] * port['rPrice']
port['Weight_adj'] = round(MV_adj.div(abs(MV_adj).sum()), 3)

print(port[['Side', 'rRAR', 'rRisk', 'rRisk_adj', 'Shares', 'Qty_adj', 'Shares_adj', 'Weight', 'Weight_adj']].groupby(
    'Side').sum())

       rRAR   rRisk  rRisk_adj  Shares  Qty_adj  Shares_adj  Weight  \
Side                                                                  
-1      6.1 -0.0483    -0.0266   -8700   1200.0     -7500.0  -0.428   
 1   -104.1 -0.0067    -0.0015   28100  -3000.0     25100.0   0.573   

      Weight_adj  
Side              
-1        -0.527  
 1         0.474  


/var/folders/21/rvw9mkxn5jz8cstyz4tllyhr0000gn/T/ipykernel_15610/3307250601.py:11: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  port['Qty_adj'] = shares_adj_short.append(shares_adj_long)


In [9]:
print(
    port[['Side', 'rRAR', 'rRisk', 'rRisk_adj', 'Shares', 'Qty_adj', 'Shares_adj', 'Weight', 'Weight_adj']].sort_values(
        by=['Side', 'rRisk_adj'], ascending=[True, False]))

       Side   rRAR   rRisk  rRisk_adj  Shares  Qty_adj  Shares_adj  Weight  \
TSLA     -1    0.1 -0.0165    -0.0000    -100    100.0         0.0  -0.007   
DIS      -1    1.1 -0.0050    -0.0037    -800    200.0      -600.0  -0.045   
NFLX     -1    1.2 -0.0059    -0.0044    -400    100.0      -300.0  -0.067   
QCOM     -1    1.4 -0.0084    -0.0071   -1900    300.0     -1600.0  -0.083   
PG       -1    2.3 -0.0125    -0.0114   -5500    500.0     -5000.0  -0.226   
MMM       1   -3.5 -0.0000    -0.0000    1600     -0.0      1600.0   0.094   
IBM       1   -1.0 -0.0000    -0.0000    1800     -0.0      1800.0   0.074   
F         1 -104.0 -0.0000    -0.0000   20800     -0.0     20800.0   0.089   
BRK-B     1    1.7 -0.0049    -0.0007    2800  -2400.0       400.0   0.247   
UPS       1    2.7 -0.0018    -0.0008    1100   -600.0       500.0   0.069   

       Weight_adj  
TSLA        0.000  
DIS        -0.049  
NFLX       -0.074  
QCOM       -0.103  
PG         -0.301  
MMM         0.138  
I

In [10]:
print('Gross Exposure', gross, 'Net Exposure', net, 'Net Beta', net_Beta, 'concentration', concentration)
gross_adj = round(abs(MV_adj).sum() / K, 3)
net_adj = round(MV_adj.sum() / abs(MV_adj).sum(), 3)
net_Beta_adj = round((MV_adj * port['Beta']).sum() / abs(MV_adj).sum(), 2)
net_pos_adj = port.loc[port['Shares_adj'] > 0, 'Shares_adj'].count() - port.loc[
    port['Shares_adj'] < 0, 'Shares_adj'].count()
print('Gross Exposure_adj', gross_adj, 'Net Exposure_adj', net_adj,
      'Net Beta_adj', net_Beta_adj, 'concentration adj', net_pos_adj)
rnet_adj = round(rMV_adj.sum() / abs(rMV_adj).sum(), 3)
rnet_Beta_adj = round((rMV_adj * port['Beta']).sum() / abs(rMV_adj).sum(), 2)
print('Gross Exposure_adj', gross_adj, 'rNet Exposure_adj', rnet_adj, 'rNet Beta_adj', rnet_Beta_adj)

Gross Exposure 3.151 Net Exposure 0.145 Net Beta 0.25 concentration 0.0
Gross Exposure_adj 2.149 Net Exposure_adj -0.052 Net Beta_adj 0.13 concentration adj 1
Gross Exposure_adj 2.149 rNet Exposure_adj -0.053 rNet Beta_adj 0.13
